In [ ]:
# 1. Install Dependencies
%pip install requests pandas geopandas mapboxgl shapely --quiet


In [ ]:
# 2. Setup / Configuration
import os
import requests

BASE_URL = 'http://linkspeed-fastapi:8000'
MAPBOX_TOKEN = 'pk.eyJ1Ijoic21hcnRtZXRhbCIsImEiOiJjbWJnbnRhbHIwMHN5MmpvaDN5M2E3dnptIn0.OOMN3laXveAb-ep2U2WqsQ'

In [ ]:
# 3. Request aggregated data
params = {
    'day': 'Wednesday',
    'period': 'pm_peak',
}

print("Sanity-check request: Get a single Link with aggregated average speed and its metadata in GeoJSON format - link_id = 16981048")
with requests.Session() as session:
    response = session.request(method='GET', url=f"{BASE_URL}/aggregates/16981048", params=params)
    print(f"Response code for sanity-check request of a single link: {response.status_code}")
    print(response.text)


print("\nMapBox visualization primary bulk request: Get all links with aggregated average speeds and their metadata in a list of GeoJSON objects.")
geojson_data = []
with requests.Session() as session:
    response = session.request(method='GET', url=f"{BASE_URL}/aggregates", params=params)
    print(f"MapBox visualization primary bulk request. (too much data to display) Response code: {response.status_code}")
    geojson_data = response.json()

In [ ]:
# 4. Visualize in Mapbox
from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops
from IPython.display import IFrame

# Assuming geojson_data is a list of Feature dicts
features = [{
    "type": "Feature",
    "geometry": f["geometry"],
    "properties": {
        "average_speed": f["properties"]["average_speed"],
        "road_name": f["properties"]["road_name"]
    }
} for f in geojson_data]

viz = ChoroplethViz(
    {
        "type": "FeatureCollection",
        "features": features
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors="Reds"),
    center=(-81.6557, 30.3322),
    zoom=11,
    line_width=1.5,
    opacity=0.8,
)

viz.create_html('map.html')
display(IFrame('map.html', width='100%', height=600))